In [2]:
import pandas as pd
import findPath
from datetime import datetime, timedelta
import predict

eta_tasks = pd.read_csv('eta_result.csv')
G = findPath.create_graph()

In [3]:
FD = 0.06

for i in range(0, eta_tasks.shape[0] - 1, 2):
# for i in range(0, 10 - 1, 2):
    # 需要的信息：'holidays', 'time_period', 'cpath', 'cost', 'x1', 'y1', 'x2', 'y2', 'speed'
    if pd.isnull(eta_tasks.iloc[i+1, 2]) == False:
        continue
    print(i)
    start = eta_tasks.iloc[i]
    end = eta_tasks.iloc[i+1]
    speed = start['speeds'] * 3.6
    holidays = start['holidays']
    time = start['time']
    parsed_time = datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ')
    time_period = parsed_time.hour
    start_X, start_Y = eval(start['coordinates'])
    end_X, end_Y = eval(end['coordinates'])
    start_id = findPath.find_nearest_node(start_X, start_Y, FD)
    end_id = findPath.find_nearest_node(end_X, end_Y, FD)
    if start_id == -1 or end_id == -1:
        continue
    path_list = findPath.get_shortest_path(G, start_id, end_id)
    delta_minutes = 0.0
    for j in range(0, len(path_list)-1):
        x1, y1 = findPath.get_coordinates(path_list[j])
        x2, y2 = findPath.get_coordinates(path_list[j+1])
        cpath, cost = findPath.get_edge_info(path_list[j],path_list[j+1])
        model_input = pd.DataFrame({
            'holidays': [holidays], 
            'time_period': [time_period], 
            'cpath': [cpath], 
            'cost': [cost], 
            'x1': [x1],
            'y1': [y1], 
            'x2': [x2], 
            'y2': [y2], 
            'speed': [speed],
            'road_cost_time': [1.14], 
        })
        delta_minutes += predict.predict(model_input)
    if type(delta_minutes) == type(0.1):
        result_time = parsed_time + timedelta(minutes=delta_minutes)
    else:
        result_time = parsed_time + timedelta(minutes=delta_minutes.item())
    result_time = result_time.replace(microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')
    eta_tasks.loc[i+1, 'time'] = result_time
eta_tasks.to_csv('eta_result.csv', index=False)
eta_tasks

4
20
34
68
112
116
128
140
156
176
190
220
242
268
270
290
332
334
336
338
340
372
376
552
554
572
578
582
584
614
628
644
666
668
682
684
726
736
746
802
804
814
836
840
846
848
870
876
900
902
926
960
1032
1046
1064
1066
1080
1082
1114
1120
1122
1142
1148
1170
1174
1176
1178
1186
1208
1216
1254
1314
1316
1318
1320
1340
1342
1372
1376
1378
1394
1438
1492
1496
1558
1590
1598
1636
1658
1668
1670
1676
1690
1712
1714
1724
1726
1736
1788
1790
1798
1800
1808
1810
1824
1828
1838
1854
1872
1874
1876
1878
1896
1916
1926
1934
1948
1950
1958
1966
2000
2010
2030
2050
2052
2070
2094
2112
2152
2166
2182
2228
2234
2264
2296
2298
2320
2336
2366
2374
2376
2428
2452
2454
2456
2502
2532
2534
2560
2566
2616
2636
2656
2710
2716
2720
2722
2740
2742
2750
2754
2764
2786
2800
2822
2870
2880
2882
2896
2898
2928
2934
2962
3004
3014
3020
3078
3080
3082
3092
3094
3106
3132
3142
3144
3148
3170
3172
3214
3236
3238
3266
3336
3344
3368
3430
3432
3434
3444
3446
3454
3468
3522
3528
3540
3546
3556
3600
3608
3668
3688
36

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays
0,126,9,2013-10-08T22:30:00Z,254,"[116.490013,39.892902]",0.000000,37.2925,0
1,133,9,2013-10-08T22:54:55Z,254,"[116.511292,39.904655]",3.079017,24.6500,0
2,147,11,2013-10-08T23:00:00Z,254,"[116.450546,39.943771]",0.000000,36.0575,0
3,164,11,2013-10-09T00:25:24Z,254,"[116.440742,40.018944]",12.543428,49.9000,0
4,171,13,2013-10-08T17:15:00Z,255,"[116.335503,39.763187]",0.000000,26.6250,0
...,...,...,...,...,...,...,...,...
8795,518214,21991,2013-10-30T21:03:47Z,4954,"[116.371025,39.84676]",7.175550,17.9425,0
8796,518315,21997,2013-10-30T22:00:00Z,4954,"[116.324913,39.938717]",0.000000,39.0125,0
8797,518348,21997,2013-10-30T22:43:32Z,4954,"[116.2817,39.945621]",11.553799,25.0725,0
8798,518408,21999,2013-10-30T00:00:00Z,4955,"[116.301109,39.985023]",0.000000,55.1025,0


In [5]:
eta_tasks[eta_tasks['time'].isnull()]

,id,traj_id,time,entity_id,coordinates,current_dis,speeds,holidays
